# Prepare Docker File

In [36]:
import os

In [37]:
cd ./flaskwebapp/

[Errno 2] No such file or directory: './flaskwebapp/'
/Users/shiwang/Desktop/Github/cloud_learn/flaskwebapp


## Define docker File

In [172]:
%%writefile Dockerfile

# Use an official Python runtime as a parent image
# If first time building, use python; else use ml_api so that cache can be utilized
#FROM python:3.6-slim
#FROM ml_api:base
FROM ml_api_with_nlp:base
    
# Copy the current directory contents into the container at /app. Different from mount
ADD . /app

# Set the working directory to /app
WORKDIR /app
RUN apt-get -y update && apt-get -y install tk-dev vim&& rm -r /var/lib/apt/lists/*

# Install any needed packages specified in requirements.txt (not local file)
RUN pip install --trusted-host pypi.python.org -r requirements.txt
RUN python -m spacy download en

# Make port 80 available to the world outside this container
EXPOSE 80

# Run app.py when the container launches
CMD ["python", "app.py"]

Overwriting Dockerfile


## Define app file

- see **GCP_App.ipynb** for detailed file

## Define requirement file

- For testing (only keep flask packages for testing purpose)

In [150]:
%%writefile requirements.txt
Flask==0.12.2
flask-wtf

Overwriting requirements.txt


- For final app

In [151]:
%%writefile requirements.txt
Flask==0.12.2
flask-wtf
matplotlib
keras
tensorflow
numpy
pandas
spacy
gensim

Overwriting requirements.txt


# Build Docker Image

## Clear all container and images

- Stop and delete all containers

In [174]:
!docker stop `docker ps -qa`
!docker rm `docker ps -qa`

f8f051b83df4
e63e10a637f5
02b02017ed00
f8f051b83df4
e63e10a637f5
02b02017ed00


- Delete all images

In [178]:
#!docker image rm -f $(docker image ls -a -q) # delete all
#!docker rmi -f $(docker images -f "dangling=true" -q) # delete <none>
#!docker image rm ml_api:base # delete one

Deleted: sha256:85e3f5572abf8a034fb11367feac07b3ec63ea10425bf10e57aa3cf51a94c9d0
Deleted: sha256:41a81727cf58a17d23ad38cdb65cc8b5a0da861c180200a318caaae5b39ae1a2
Deleted: sha256:ceef5a11a318756554add647b40acbbb3920d31ee43335034cbb79c8b06a8d44
Deleted: sha256:5a2d34edbb938cf25553e7749f0a2d9bba17d0b8e3dd21d31b2bdb073b200ab2
Deleted: sha256:2af942a056a75d274fef35a90ccd799520d759e580d4066b5b3be843b9f3e023
Deleted: sha256:3f8eb0429d0908e2bd46bd27a981b6daa7ad84734c446466dce40587983542ae
Deleted: sha256:5f8268bb109723f3b32210af842e7e1b12e86cc9146a070ee24705998ebea700
Deleted: sha256:26c44b5ab87099c012f425d895743af71ea86db1d5cf78aa477ee8977fde22a4
Deleted: sha256:cb2345f6cec50aefbfa2236cc65fc4007137a66b4c71ae3ed0c968781617f3fe
Deleted: sha256:79d91d9e23506299f73ddd96cfa7b6d7c119ce84e70b937d9276c82dfcbdb27d
Deleted: sha256:8d48297022bdd6e0909f90e620dfa2257378c9e7a1c4b2bb605d96d4da085cc6


- List all images

In [176]:
!docker image ls

REPOSITORY                         TAG                 IMAGE ID            CREATED             SIZE
ml_api_with_nlp                    base                b8084184c499        49 seconds ago      2.28GB
ml_api                             latest              85e3f5572abf        4 minutes ago       2.28GB
<none>                             <none>              28985798dab7        19 minutes ago      2.28GB
ml_api_no_nlp                      base                04c5b84c0c90        30 hours ago        1.53GB
gcr.io/keen-hangar-197817/ml_api   latest              04c5b84c0c90        30 hours ago        1.53GB
kingstone010148/ml_api             latest              a7ed42a1a051        4 days ago          207MB
python                             3.6-slim            8aa59c33d0d5        2 weeks ago         138MB


## Build docker

In [179]:
# For first time building base image

In [ ]:
!docker build -t ml_api:latest .
!docker tag ml_api:latest ml_api:base

In [157]:
!docker image ls

REPOSITORY                         TAG                 IMAGE ID            CREATED             SIZE
ml_api                             base                d827750ebcf1        3 seconds ago       2.32GB
ml_api                             latest              d827750ebcf1        3 seconds ago       2.32GB
ml_api_no_nlp                      base                04c5b84c0c90        About an hour ago   1.53GB
gcr.io/keen-hangar-197817/ml_api   latest              04c5b84c0c90        About an hour ago   1.53GB
kingstone010148/ml_api             latest              a7ed42a1a051        3 days ago          207MB
python                             3.6-slim            8aa59c33d0d5        2 weeks ago         138MB


## Run docker on local

- Start container

In [158]:
# -d: background
# -v: mount local to docker
# -p: localhost -> docker port
# -it: interactivae
!docker run -d -it -p 5000:80 ml_api
# -v ~/Desktop/Github/cloud_learn/flaskwebapp:/app 

94ee3299cbe1478481d01abff8a8f5e114f80b471d7ac25b3941a2f49838034e


- Check running container

In [159]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED                  STATUS              PORTS                  NAMES
94ee3299cbe1        ml_api              "python app.py"     Less than a second ago   Up 1 second         0.0.0.0:5000->80/tcp   goofy_kirch


- Run commands inside container when necessary

`docker exec -it 390d72691510 /bin/bash`

- Confirm results

In [160]:
!curl http://0.0.0.0:5000/api/test

{
  "result": "ok"
}


- Browser: http://localhost:5000/api/test

- Stop all containers

In [161]:
!docker stop `docker ps -qa`
!docker rm `docker ps -qa`

94ee3299cbe1
94ee3299cbe1


# (optional) Push image to Docker Hub

- Push image to docker hub

In [19]:
!docker image rm kingstone010148/ml_api
!docker tag ml_api kingstone010148/ml_api
!docker push kingstone010148/ml_api

The push refers to repository [docker.io/kingstone010148/ml_api]

a59d6a1e: Preparing 
7f3f9e0a: Preparing 
6fe8f7ab: Preparing 
6aa6e624: Preparing 
7ba54998: Preparing 
0e7d22be: Preparing 
f9544e4c: Pushed   58.44MB/55.27MBPushing  17.66MB/69.07MBPushing  38.97MB/55.27MBPushing  49.83MB/55.27MBlatest: digest: sha256:8b534e4801af958d4039ea873e875d4db0191b0a09402bffb832652374b0097b size: 1793


- Run remote image

In [20]:
!docker run -d -p 5000:80 kingstone010148/ml_api

7040be03a0d270ecd786871eaf960f7689ce1cbe2dbfc7d273fed536da475dad
docker: Error response from daemon: driver failed programming external connectivity on endpoint hopeful_visvesvaraya (1b29357de82ed89f9d325f941fc1d0bf806daaec54e69ff3b8885fdefdbc6f97): Bind for 0.0.0.0:5000 failed: port is already allocated.


In [22]:
!curl http://0.0.0.0:5000/api/test

{
  "result": "ok"
}


In [23]:
!docker stop `docker ps -qa`
!docker rm `docker ps -qa`

7040be03a0d2
9ab831ac3c2e
7040be03a0d2
9ab831ac3c2e


# Push image to GCP Google Container Registry

In [180]:
!docker image rm gcr.io/keen-hangar-197817/ml_api
!docker tag ml_api:latest gcr.io/keen-hangar-197817/ml_api

Untagged: gcr.io/keen-hangar-197817/ml_api:latest
Untagged: gcr.io/keen-hangar-197817/ml_api@sha256:dfd78e01b4b099e8d3704388526829739aa57419d3349606933765b1b18a5735


In [125]:
!docker image ls

REPOSITORY                         TAG                 IMAGE ID            CREATED             SIZE
ml_api                             base                04c5b84c0c90        26 minutes ago      1.53GB
ml_api                             latest              04c5b84c0c90        26 minutes ago      1.53GB
gcr.io/keen-hangar-197817/ml_api   latest              04c5b84c0c90        26 minutes ago      1.53GB
kingstone010148/ml_api             latest              a7ed42a1a051        3 days ago          207MB
python                             3.6-slim            8aa59c33d0d5        2 weeks ago         138MB


- Push to Google Container Registry

In [119]:
!docker push gcr.io/keen-hangar-197817/ml_api

The push refers to repository [gcr.io/keen-hangar-197817/ml_api]

6f12ee3e: Preparing 
170a9f4e: Preparing 
2bcbaa73: Preparing 
21ef036a: Preparing 
98dc0494: Preparing 
564ee966: Preparing 
17ae2d4d: Preparing 
81b8fe9b: Preparing 
dc1f2bb8: Preparing 
a470e4a2: Preparing 
6009a73e: Preparing 
8f9a8504: Preparing 
2a441292: Preparing 
08a876df: Preparing 
6fe8f7ab: Preparing 
6aa6e624: Preparing 
7ba54998: Preparing 
0e7d22be: Preparing 


6009a73e: Pushing    147MB/333.1MB18APushing  53.46MB/57.8MBPushing  67.18MB/333.1MB2a441292: Pushing  409.4MB/739.1MB

2a441292: Pushed   756.7MB/739.1MBPushing  476.8MB/739.1MBPushing  518.4MB/739.1MBlatest: digest: sha256:dfd78e01b4b099e8d3704388526829739aa57419d3349606933765b1b18a5735 size: 4309


In [124]:
!gcloud container images list-tags gcr.io/keen-hangar-197817/ml_api

DIGEST        TAGS    TIMESTAMP
dfd78e01b4b0  latest  2018-08-15T10:16:49


In [121]:
#!gcloud container images delete gcr.io/keen-hangar-197817/ml_api@[IMAGE_DIGEST]

- Start VM instance with container
  - Reference: Create VM instance on GCP and deploy the container: https://cloud.google.com/compute/docs/containers/deploying-containers
  - Reference: Copy local files to VM: `!gcloud compute scp app.py instance-7:~/`
  - Reference: if mount: `--container-mount-host-path mount-path=/app,host-path=/,mode=rw`

In [126]:
!gcloud beta compute instances create-with-container instance-ml-api \
    --zone=us-east1-b \
    --container-image gcr.io/keen-hangar-197817/ml_api \
    --tags=mlapi,http-server,https-server

Created [https://www.googleapis.com/compute/beta/projects/keen-hangar-197817/zones/us-east1-b/instances/instance-ml-api].
NAME             ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
instance-ml-api  us-east1-b  n1-standard-1               10.142.0.2   35.237.59.110  RUNNING


# Connect to GCP VM by external IP

In [128]:
!curl http://35.237.59.110/api/test

{
  "result": "ok"
}


## Reference - 
- https://cloud.google.com/compute/docs/containers/deploying-containers
- https://cloud.google.com/compute/docs/containers/configuring-options-to-run-containers
- https://cloud.google.com/container-registry/docs/pushing-and-pulling
- https://cloud.google.com/container-registry/docs/managing